Day 11

this is essentially the game of life


In [12]:
import import_ipynb
import util
from copy import deepcopy

def print_map(seats):
    for row in seats:
        print(row)

def solution1():
    seat_map = [list(row) for row in util.input_reader("day11.input", 1)]
    seat_map2 = deepcopy(seat_map)
    directions = [[0, 1], [1, 0], [0, -1], [-1, 0], [1, 1], [-1, -1], [1, -1], [-1, 1]]
    num_rows = len(seat_map)
    num_cols = len(seat_map[0])

    while True:
        something_changed = False
        for r in range(num_rows):
            for c in range(num_cols):
                if seat_map[r][c] == ".":
                    continue
                s = sum([1 if r+d[0] >= 0 and r+d[0] < num_rows and c+d[1] >= 0 and c+d[1] < num_cols and seat_map[r+d[0]][c+d[1]] == "#" else 0 for d in directions])
                if seat_map[r][c] == "L" and s == 0:
                    seat_map2[r][c] = "#"
                    something_changed = True
                elif seat_map[r][c] == "#" and s >= 4:
                    seat_map2[r][c] = "L"
                    something_changed = True
        seat_map = deepcopy(seat_map2)
        if not something_changed:
            break
    
    count = 0
    for row in seat_map:
        for s in row:
            count += 1 if s == "#" else 0
    return count

solution1()

2222

same idea but instead of a fixed direction map, we need to precompute directions for each seat and save that

In [41]:
def build_directions_map(seat_map, num_rows, num_cols):
    directions = [[0, 1], [1, 0], [0, -1], [-1, 0], [1, 1], [-1, -1], [1, -1], [-1, 1]]
    directions_map = deepcopy(seat_map)
    for r in range(num_rows):
        for c in range(num_cols):
            neighbor_directions = []
            if seat_map[r][c] == ".":
                directions_map[r][c] = neighbor_directions
                continue

            for direction in directions:
                d = deepcopy(direction)
                while bounds_check(r, c, d, num_rows, num_cols):
                    if seat_map[r+d[0]][c+d[1]] == "#" or seat_map[r+d[0]][c+d[1]] == "L":
                        neighbor_directions.append(d)
                        break
                    d[0] += direction[0]
                    d[1] += direction[1]
            directions_map[r][c] = neighbor_directions
    return directions_map

def bounds_check(r, c, d, num_rows, num_cols):
    return r+d[0] >= 0 and r+d[0] < num_rows and c+d[1] >= 0 and c+d[1] < num_cols

def solution2():
    seat_map = [list(row) for row in util.input_reader("day11.input", 1)]
    seat_map2 = deepcopy(seat_map)
    num_rows = len(seat_map)
    num_cols = len(seat_map[0])

    directions_map = build_directions_map(seat_map, num_rows, num_cols)

    while True:
        something_changed = False
        for r in range(num_rows):
            for c in range(num_cols):
                if seat_map[r][c] == ".":
                    continue
                neighbor_directions = directions_map[r][c]
                s = sum([1 if seat_map[r+d[0]][c+d[1]] == "#" else 0 for d in neighbor_directions])
                if seat_map[r][c] == "L" and s == 0:
                    seat_map2[r][c] = "#"
                    something_changed = True
                elif seat_map[r][c] == "#" and s >= 5:
                    seat_map2[r][c] = "L"
                    something_changed = True
                else:
                    seat_map2[r][c] = seat_map[r][c]
        seat_map, seat_map2 = seat_map2, seat_map
        if not something_changed:
            break
    return sum([sum([1 if s == "#" else 0 for s in row]) for row in seat_map])
solution2()

2032

let's do some profiling. hypothesis, matrix swapping instead of deepcopy will be much faster. This time is the old time (deepcopy)

In [28]:
%timeit solution2()

2.01 s ± 14.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


this is matrix swapping

In [42]:
%timeit solution2()

1.63 s ± 18.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


seems like there's a diff